# Assignment 1, Advanced Analytics in Business
# Jack Heller, Baris Aksoy, Medha Hegde, Aleksandra Zdravkovic, Group 31

Here we start assignment 1 of the course.  Our goal is to identify the most likely customers who will leave the bank.  This is done using a variety of features that include information about the accounts of customers and their balances, as well as the activity of their accounts and any loans they may have with the bank and background information on the customer (education, family, etc.).  This data is also provided for the trailing three months so that changes in patterns can be observed.

## Problem Defintion

Due to the limited resources of the bank, the 250 clients that are predicted as most likely to churn will be contacted by the bank in an attempt to retain them.  This means that our goal is not to effectively predict which customers will not churn as it is to find which customers will churn.  Being at a large bank, customers will leave every month.  However, we need to make sure the staff's time is used most effectively which means not wasting their time with customers who will not leave.  This means that a measure like accuracy or AUC would not be appropriate.  Instead, we should count the number of true positives in the top 250.  This can be done using precision.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve, roc_auc_score
from matplotlib import pyplot as plt
import datetime
from sklearn import preprocessing
from sklearn import metrics, model_selection
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import missingno as msno
from sklearn.model_selection import train_test_split
from category_encoders.woe import WOEEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
curDat = pd.read_csv('train_month_3_with_target.csv')
mon2Dat = pd.read_csv('train_month_2.csv')
mon1Dat = pd.read_csv('train_month_1.csv')

# Exploratory Data Analysis

We start by investigating what data we have and how it can be transformed into more valuable features.

For a full description of the variables please visit: http://seppe.net/aa/assignment1/

Immediately we can see many binary variables describing the customer's relationship with the bank.  These will be very useful not on their own but when combined with trailing data.  For example, if a customer was very active and then stopped using the account, this is a bad sign.  Also if a customer has recently paid off a loan or cancelled insurance, they may be preparing to leave.

# Baris insert info about balances here

Other variables have a different story to tell.  For example, the variables, 'customer_since_bank' is given as a year and month.  This information is very useful as a customer who has been with the bank for significant time is less likely to leave.  However, this needs to be converted to time as customer in either months or years to allow an easier interpretation of this decaying likelihood.

We have another set of variables that need to be featurized in another way.  These are categorical variables where some are ordinal and some are not.  For example, occupation codes are not ordinal.  However, some are certainly more likely to churn than others.  This is true of categories such as customer children as well.  We also have some information on the customer's education level.

We also have information on the customer's postal codes.  This poses an interesting challenge as some postal codes appear to have a much higher likelihood of churning than others.  However, some post codes have so few customers that it is not completely reliable and creates a very high dimensional problem as there are many post codes in Belgium.  Fortunately, these 4 digit numbers make up larger areas so we are able to easily aggregate these codes into larger areas by only using the first two numbers.  For a visualization of this, see the map of 2 digit postal codes below.

<img src="2_digit_postcode_belgique.png" width="400" height="340">
https://en.wikipedia.org/wiki/Postal_codes_in_Belgium

When investigating the target data, is it important to note that this data is extremely unbalanced.  With only about 3% of the whole dataset being churning clients.

In [3]:
curDat.describe()

,homebanking_active,has_homebanking,has_insurance_21,has_insurance_23,has_life_insurance_fixed_cap,has_life_insurance_decreasing_cap,has_fire_car_other_insurance,has_personal_loan,has_mortgage_loan,has_current_account,...,bal_savings_account_starter,bal_current_account_starter,visits_distinct_so,visits_distinct_so_areas,customer_gender,customer_postal_code,customer_occupation_code,customer_self_employed,customer_education,target
count,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,...,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,63697.000000,61695.000000,63697.000000,16572.000000,63697.000000
mean,0.215191,0.280939,0.095028,0.009953,0.002747,0.111779,0.318053,0.041619,0.098089,0.500809,...,57.641176,30.320894,1.230199,1.042608,1.486444,5577.261959,8.773531,0.087021,2.463734,0.030033
std,0.410958,0.449462,0.293256,0.099270,0.052344,0.315097,0.465724,0.199718,0.297438,0.500003,...,892.959859,407.877892,0.501498,0.224991,0.499820,3020.064554,1.131453,0.281869,1.520309,0.170679
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-330.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,2650.000000,9.000000,0.000000,2.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,4877.000000,9.000000,0.000000,2.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.000000,1.000000,2.000000,8750.000000,9.000000,0.000000,3.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,24050.000000,19790.000000,7.000000,6.000000,2.000000,9992.000000,9.000000,1.000000,6.000000,1.000000


In [4]:
curDat.columns

Index(['client_id', 'homebanking_active', 'has_homebanking',
       'has_insurance_21', 'has_insurance_23', 'has_life_insurance_fixed_cap',
       'has_life_insurance_decreasing_cap', 'has_fire_car_other_insurance',
       'has_personal_loan', 'has_mortgage_loan', 'has_current_account',
       'has_pension_saving', 'has_savings_account',
       'has_savings_account_starter', 'has_current_account_starter',
       'bal_insurance_21', 'bal_insurance_23', 'cap_life_insurance_fixed_cap',
       'cap_life_insurance_decreasing_cap', 'prem_fire_car_other_insurance',
       'bal_personal_loan', 'bal_mortgage_loan', 'bal_current_account',
       'bal_pension_saving', 'bal_savings_account',
       'bal_savings_account_starter', 'bal_current_account_starter',
       'visits_distinct_so', 'visits_distinct_so_areas', 'customer_since_all',
       'customer_since_bank', 'customer_gender', 'customer_birth_date',
       'customer_postal_code', 'customer_occupation_code',
       'customer_self_employed',

### Data Completeness

One challenge we did have is that not all customers fill in every field and we have incomplete information about every customer.  Removing these customers from the dataset is not a good option as this will continue to be a problem in the test set.  However, we can investigate how these missing data points impact their likelihood to churn.

In [5]:
curDat.isna().sum()

client_id                                0
homebanking_active                       0
has_homebanking                          0
has_insurance_21                         0
has_insurance_23                         0
has_life_insurance_fixed_cap             0
has_life_insurance_decreasing_cap        0
has_fire_car_other_insurance             0
has_personal_loan                        0
has_mortgage_loan                        0
has_current_account                      0
has_pension_saving                       0
has_savings_account                      0
has_savings_account_starter              0
has_current_account_starter              0
bal_insurance_21                         0
bal_insurance_23                         0
cap_life_insurance_fixed_cap             0
cap_life_insurance_decreasing_cap        0
prem_fire_car_other_insurance            0
bal_personal_loan                        0
bal_mortgage_loan                        0
bal_current_account                      0
bal_pension

We start by evaluating how a customers lack of information effects their likelihood of churning.  We can see that while churning is pretty similar among all education levels with the exception of 6, customers who did not fill in their information are much less likely to churn.  We can also see that the vast majority of customers did not include their education level information.

In [6]:
# Can be seen that customer education na is a big effect
curDat['customer_education'] = curDat['customer_education'].fillna('nan')
print(curDat.groupby('customer_education', as_index=False)['target'].mean())
print(curDat.groupby('customer_education', as_index=False)['target'].count())

  customer_education    target
0                0.0  0.057851
1                1.0  0.066593
2                2.0  0.063693
3                3.0  0.062213
4                4.0  0.057471
5                5.0  0.059593
6                6.0  0.032154
7                nan  0.018992
  customer_education  target
0                0.0    2178
1                1.0    1802
2                2.0    4506
3                3.0    5015
4                4.0     696
5                5.0    2064
6                6.0     311
7                nan   47125


We can see that when we look at occupation codes, almost all of our customers have occupation 9.  While evaluating solely on percentage churning by occupation it shows that some don't churn.  However, with so few observations, it is difficult to say definitively that they really are different.

In [7]:
curDat['customer_occupation_code'] = curDat['customer_occupation_code'].fillna('nan')
print(curDat.groupby('customer_occupation_code', as_index=False)['target'].mean())
print(curDat.groupby('customer_occupation_code', as_index=False)['target'].count())

   customer_occupation_code    target
0                       0.0  0.026128
1                       1.0  0.000000
2                       2.0  0.000000
3                       3.0  0.100000
4                       4.0  0.041489
5                       5.0  0.071895
6                       6.0  0.054645
7                       7.0  0.038462
8                       8.0  0.037736
9                       9.0  0.029217
10                      nan  0.038462
   customer_occupation_code  target
0                       0.0     421
1                       1.0      24
2                       2.0       7
3                       3.0      10
4                       4.0    1639
5                       5.0     153
6                       6.0     183
7                       7.0     104
8                       8.0     318
9                       9.0   58836
10                      nan    2002


When doing the same evaluation for children, we see that customers with one baby, preschool children, young children, and those who simply answered yes are much more likely to churn.  They are also a small population compared to the overall population.  On the other side, those who did not answer or answered no make up almost all of the customers and have very similar likelihoods of churning.

In [8]:
curDat['customer_children'] = curDat['customer_children'].fillna('nan')
print(curDat.groupby('customer_children', as_index=False)['target'].mean())
print(curDat.groupby('customer_children', as_index=False)['target'].count())

  customer_children    target
0        adolescent  0.032464
1           grownup  0.035115
2            mature  0.025366
3               nan  0.028805
4                no  0.023202
5           onebaby  0.057981
6         preschool  0.063738
7               yes  0.065089
8             young  0.051659
  customer_children  target
0        adolescent    3912
1           grownup    1908
2            mature    4849
3               nan   23364
4                no   22886
5           onebaby    1466
6         preschool    2322
7               yes     338
8             young    2652


We do one more analysis in this fashion on customer relationships. Here we see that being in a relationship appears to have very little effect on if they will churn and not answering is also not a strong indicator.

In [9]:
curDat['customer_relationship'] = curDat['customer_relationship'].fillna('nan')
print(curDat.groupby('customer_relationship', as_index=False)['target'].mean())
print(curDat.groupby('customer_relationship', as_index=False)['target'].count())

  customer_relationship    target
0                couple  0.031317
1                   nan  0.029935
2                single  0.026468
  customer_relationship  target
0                couple   36179
1                   nan   14899
2                single   12619


# Why did we drop postal code?

Here we featurize the data as described.  We:
- Remove Client ID
- Create a feature to identify if they were homebanking 3 months ago and stopped
- If their savings account balance changed
- If the customer has a current checking account
- If there was a change in the customer's loan or mortgage status
- Postal code is dropped as it will be added back in later
- We use a median imputation for how long the customer has been with the bank as this data is rarely NA.

In [10]:
curDat = pd.read_csv('train_month_3_with_target.csv')

curDat = curDat.drop(['client_id'], axis = 1)

homeChange = curDat['homebanking_active'] - mon1Dat['homebanking_active']
curDat.insert(1, "homebank_change", homeChange)

saveBalChange = curDat['bal_savings_account'] - mon1Dat['bal_savings_account']
curDat.insert(1, "saveBalChange", saveBalChange)

curActChange = curDat['has_current_account'] - mon1Dat['has_current_account']
curDat.insert(1, "has_current_account_change", curActChange)

perLoanChange = mon1Dat['has_personal_loan'] - curDat['has_personal_loan']
curDat.insert(1, 'change_personal_loan', perLoanChange)

homeLoanChange = mon1Dat['has_mortgage_loan'] - curDat['has_mortgage_loan']
curDat.insert(1, 'change_mortgage_loan', homeLoanChange)

curDat['customer_occupation_code'] = curDat['customer_occupation_code'].fillna(10)
dumOccCode = pd.get_dummies(curDat['customer_occupation_code'], prefix = 'occ_code')
curDat = pd.concat([dumOccCode, curDat], axis = 1)
curDat = curDat.drop(['customer_occupation_code'], axis = 1)

curDat['customer_education'] = curDat['customer_education'].fillna(7)

curDat['customer_children'] = curDat['customer_children'].fillna('nan')
dumChilCode = pd.get_dummies(curDat['customer_children'], prefix = 'child_code')
curDat = pd.concat([dumChilCode, curDat], axis = 1)
curDat = curDat.drop(['customer_children'], axis = 1)

curDat = curDat.drop(['customer_relationship'], axis = 1)

curDat['customer_since_all'] = (datetime.datetime.today() - pd.to_datetime(curDat['customer_since_all'])).dt.days
curDat['customer_since_bank'] = (datetime.datetime.today() - pd.to_datetime(curDat['customer_since_bank'])).dt.days
curDat['customer_birth_date'] = (datetime.datetime.today() - pd.to_datetime(curDat['customer_birth_date'])).dt.days

curDat['customer_since_all'] = curDat['customer_since_all'].fillna(curDat['customer_since_all'].median())
curDat['customer_since_bank'] = curDat['customer_since_bank'].fillna(curDat['customer_since_bank'].median())

### Class Imbalance

We next work to deal with the class imbalance problem.  We will do this by using random oversampling.  We do this at this point as to make sure we have a validation set to evaluate on for tuning and we only create observation training reliant features on the training set.  We found that oversampling was a key step in this process.

# Shouldn't we do the train test split before EDA?

In [11]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_train = curDat.iloc[:,:-1]
y_train = curDat.iloc[:,-1]
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8)

### Post Code Featurization

Now that we have a training set, we can featurize this by taking the likelihood of churning by post code and creating a dictionary.  We then replace the post codes with these likelihoods.

In [12]:
#postDatFrame = pd.concat([Xtrain, ytrain], axis = 1)
#meanPost = pd.Series(postDatFrame.groupby('customer_postal_code')['target'].mean())
#countPost = postDatFrame.groupby('customer_postal_code')['target'].count()
#postDict = pd.Series(meanPost,index=meanPost.index).to_dict()

#Xtrain.replace({'customer_postal_code': postDict},inplace=True)
#Xval.replace({'customer_postal_code': postDict},inplace=True)

We now scale all the training data and perform the same transform on our validation set.

In [13]:
scaler = preprocessing.StandardScaler().fit(Xtrain)

Xtrain = scaler.transform(Xtrain)

Xval = scaler.transform(Xval)

# Modeling

### Logistic Regression

The first model we investigated was a basic logistic regression.  This allows us to set a benchmark for how different models perform and their precision at different thresholds.

In [14]:
logReg = LogisticRegression()
logReg.fit(Xtrain, ytrain)
predicted = logReg.predict_proba(Xval)

In [15]:
# Method to find percentage correctly identified in top numberEval predictions
def pctCorrectTop(numberEval, true, preds):
    idx = (-preds[:,1]).argsort()[:numberEval]
    yval2 = yval
    yval2 = yval2.reset_index()
    return(yval2.iloc[idx,:]['target'].sum() / numberEval)

In [16]:
logReg10 = pctCorrectTop(10, yval, predicted)
logReg50 = pctCorrectTop(50, yval, predicted)
logReg100 = pctCorrectTop(100, yval, predicted)
logReg250 = pctCorrectTop(250, yval, predicted)
print(logReg10)
print(logReg50)
print(logReg100)
print(logReg250)

0.1
0.14
0.15
0.156


In [17]:
rfClass = RandomForestClassifier()
rfClass.fit(Xtrain, ytrain)
val_predict=rfClass.predict_proba(Xval)
rf10 = pctCorrectTop(10, yval, val_predict)
rf50 = pctCorrectTop(50, yval, val_predict)
rf100 = pctCorrectTop(100, yval, val_predict)
rf250 = pctCorrectTop(250, yval, val_predict)
print(rf10)
print(rf50)
print(rf100)
print(rf250)

0.0
0.1
0.08
0.124


In [18]:
gradBoost = GradientBoostingClassifier()
gradBoost.fit(Xtrain, ytrain)
gradPreds = gradBoost.predict_proba(Xval)
gb10 = pctCorrectTop(10, yval, gradPreds)
gb50 = pctCorrectTop(50, yval, gradPreds)
gb100 = pctCorrectTop(100, yval, gradPreds)
gb250 = pctCorrectTop(250, yval, gradPreds)
print(gb10)
print(gb50)
print(gb100)
print(gb250)

0.2
0.16
0.18
0.152


## Model Comparison and Results

Here we can see that at every level, gradient boosting trees performed the best on the validation set.  For this reason, it appears to be the best model to proceed with.  We can also see the precision as various amounts of observations taken.  For gradient boosting, it may be better to take less than the top 250, the precision is much better for the top 100.  Of course, more churners were found in the top 250 but the benefit of pursuing fewer clients is that less resources and investment is required and there is a greater ROI.

In [19]:
npDf = np.array([('Logistic Regression', logReg10, logReg50, logReg100, logReg250),
                ('Random Forest', rf10, rf50, rf100, rf250),
                ('Gradient Boosting Tree', gb10, gb50, gb100, gb250)])
resultsDf = pd.DataFrame(npDf, columns = ['Model', 'TP 10', 'TP 50', 'TP 100', 'TP 250'])
resultsDf

,Model,TP 10,TP 50,TP 100,TP 250
0,Logistic Regression,0.1,0.14,0.15,0.156
1,Random Forest,0.0,0.1,0.08,0.124
2,Gradient Boosting Tree,0.2,0.16,0.18,0.152


# Creation of Test Set Predictions

Here we create our test set predictions

In [20]:
test3 = pd.read_csv('test_month_3.csv')
test1 = pd.read_csv('test_month_1.csv')

ids = test3['client_id']
test3 = test3.drop(['client_id'], axis = 1)

homeChange = test3['homebanking_active'] - test1['homebanking_active']
test3.insert(1, "homebank_change", homeChange)

saveBalChange = test3['bal_savings_account'] - test1['bal_savings_account']
test3.insert(1, "saveBalChange", saveBalChange)

curActChange = test3['has_current_account'] - test1['has_current_account']
test3.insert(1, "has_current_account_change", curActChange)

perLoanChange = test1['has_personal_loan'] - test3['has_personal_loan']
test3.insert(1, 'change_personal_loan', perLoanChange)

homeLoanChange = test1['has_mortgage_loan'] - test3['has_mortgage_loan']
test3.insert(1, 'change_mortgage_loan', homeLoanChange)

test3['customer_occupation_code'] = test3['customer_occupation_code'].fillna(10)
dumOccCode = pd.get_dummies(test3['customer_occupation_code'], prefix = 'occ_code')
test3 = pd.concat([dumOccCode, test3], axis = 1)
test3 = test3.drop(['customer_occupation_code'], axis = 1)

test3['customer_education'] = test3['customer_education'].fillna(7)

test3['customer_children'] = test3['customer_children'].fillna('nan')
dumChilCode = pd.get_dummies(test3['customer_children'], prefix = 'child_code')
test3 = pd.concat([dumChilCode, test3], axis = 1)
test3 = test3.drop(['customer_children'], axis = 1)

test3 = test3.drop(['customer_relationship'], axis = 1)

test3['customer_since_all'] = (datetime.datetime.today() - pd.to_datetime(test3['customer_since_all'])).dt.days
test3['customer_since_bank'] = (datetime.datetime.today() - pd.to_datetime(test3['customer_since_bank'])).dt.days
test3['customer_birth_date'] = (datetime.datetime.today() - pd.to_datetime(test3['customer_birth_date'])).dt.days

test3['customer_since_all'] = test3['customer_since_all'].fillna(curDat['customer_since_all'].median())
test3['customer_since_bank'] = test3['customer_since_bank'].fillna(curDat['customer_since_bank'].median())

In [21]:
xtest = scaler.transform(test3)

finalPredictions = pd.Series(gradBoost.predict_proba(xtest)[:,1])

fpredict=pd.concat([ids,finalPredictions],axis=1)
fpredict.to_csv('test_predictions',index=False,header=False)

# Reflection

When we see the second half of the test set, we had 35 true positives out of the top 250.  This is encouraging performance after seeing 36 true positives on the first test set.  We had 39 true positives on the validation set so while we did have some degradation in performance, it was acceptable.  We were very careful to not continuously submit models to the test set but we instead performed tuning and different approaches on the validation set.  This is why we saw a similar performance.

While the definition of the target variable is not bad the way it is, it may be helpful to we see from our validation set that the true positive rate is better when only taking the top 100 candidates.  Our recommendation would be that the bank perform a cost benefit analysis to evaluate if the additional resources required to contact 150 more people is worth the payoff of finding the extra people.  This could be done through a breakeven analysis of what is an acceptable true positive rate and then finding a good threshold.